# Westminster Ground Truth Analysis

This notebook demonstrates the complete workflow for creating orthomosaics from DJI drone imagery and evaluating their accuracy:

1. **Data Loading**: Load images, GCPs, and DJI metadata
2. **Feature Detection & Matching**: Detect and match features across images
3. **Camera Pose Estimation**: Estimate initial camera poses
4. **Bundle Adjustment**: Refine poses to minimize reprojection error
5. **Orthomosaic Creation**: Generate orthomosaics with and without GCPs
6. **Basemap Comparison**: Download basemaps and quantify absolute accuracy
7. **Visualization**: Visualize matches, errors, and results


## Setup: Install Dependencies

First, install the required packages if they're not already installed.


In [26]:
# Install required packages
import subprocess
import sys
from pathlib import Path

# Try to install from requirements.txt first
requirements_file = Path("requirements.txt")
if requirements_file.exists():
    print("Installing packages from requirements.txt...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", str(requirements_file)])
    print("✓ Packages installed from requirements.txt")
else:
    # Fallback: install packages individually
    print("requirements.txt not found. Installing packages individually...")
    packages = [
        "numpy>=1.24.0",
        "opencv-python>=4.8.0",
        "scipy>=1.11.0",
        "scikit-image>=0.21.0",
        "rasterio>=1.3.0",
        "pillow>=10.0.0",
        "matplotlib>=3.7.0",
        "pandas>=2.0.0",
        "pyproj>=3.6.0",
        "shapely>=2.0.0",
        "requests>=2.31.0",
        "tqdm>=4.66.0",
        "exifread>=3.0.0",
        "utm>=0.7.0"
    ]
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    print("✓ All packages installed")

print("\nSetup complete!")


Installing packages from requirements.txt...
✓ Packages installed from requirements.txt

Setup complete!


## Imports


In [27]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Add package to path
sys.path.insert(0, str(Path.cwd()))

from westminster_ground_truth_analysis import (
    GCPParser,
    DJIMetadataParser,
    OrthomosaicPipeline,
    download_basemap,
    compare_orthomosaic_to_basemap,
    visualize_matches,
    visualize_reprojection_errors,
    visualize_camera_poses,
    create_match_quality_report
)

# Set up paths
data_dir = Path("/Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25")
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

print("Setup complete!")


Setup complete!


## 1. Load Ground Control Points


In [28]:
# Parse GCP file
gcp_file = data_dir / "25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv"
gcp_parser = GCPParser(str(gcp_file))

gcps = gcp_parser.get_gcps()
print(f"Loaded {len(gcps)} ground control points")

# Display first few GCPs
for gcp in gcps[:5]:
    print(f"  {gcp.name}: X={gcp.x:.2f}, Y={gcp.y:.2f}, Z={gcp.z:.2f}")

# Get bounds
min_x, min_y, max_x, max_y = gcp_parser.get_bounds()
print(f"\nGCP Bounds: X=[{min_x:.2f}, {max_x:.2f}], Y=[{min_y:.2f}, {max_y:.2f}]")


Loaded 23 ground control points
  GCP1: X=5450945.53, Y=506914.12, Z=77.45
  GCP2: X=5450730.01, Y=506657.79, Z=79.22
  GCP3: X=5450480.01, Y=506577.77, Z=59.40
  GCP4: X=5450578.63, Y=506765.03, Z=65.59
  GCP5: X=5450715.96, Y=506926.13, Z=63.10

GCP Bounds: X=[5450109.82, 5450992.66], Y=[506577.77, 507315.01]


## 2. Process First Dataset (DJI_202510060955_017_25-3288)


In [29]:
# Setup for first dataset
dataset1_dir = data_dir / "DJI_202510060955_017_25-3288"

# Try to parse DJI metadata
dji_metadata1 = DJIMetadataParser(str(dataset1_dir))

# Create pipeline without GCPs first
pipeline1_no_gcp = OrthomosaicPipeline(
    image_dir=str(dataset1_dir),
    output_dir=str(output_dir / "dataset1_no_gcp"),
    feature_detector="sift",
    max_features=5000,
    match_ratio=0.7,
    use_gcps=False,
    dji_metadata=dji_metadata1
)

print("Processing dataset 1 (without GCPs)...")
output1_no_gcp = pipeline1_no_gcp.run_full_pipeline(output_name="dataset1_no_gcp")


Parsing timestamp file: /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25/DJI_202510060955_017_25-3288/DJI_202510060955_017_25-3288_Timestamp.MRK
Parsing navigation file: /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25/DJI_202510060955_017_25-3288/DJI_202510060955_017_25-3288_PPKNAV.nav
Processing dataset 1 (without GCPs)...
Starting Orthomosaic Pipeline
Found 543 images


Loading images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 543/543 [01:12<00:00,  7.48it/s]


Loaded 543 images
Loading features from cache: outputs/dataset1_no_gcp/cache/features.pkl
Loaded features for 543 images
Loading matches from cache: outputs/dataset1_no_gcp/cache/matches.pkl
Loaded 100 matches
Estimating initial camera poses...
Initialized poses for 2 cameras
Performing bundle adjustment...
Bundle adjustment: 1905 3D points, 3810 observations, 2 cameras
Initial mean reprojection error: 0.33 pixels
Note: Using simplified bundle adjustment. For production, consider OpenSfM or COLMAP.

Reprojection Errors:
  Mean: 0.33 pixels
  Median: 0.06 pixels
  Max: 382.70 pixels
Creating orthomosaic (resolution: 0.1m/pixel)...
Orthomosaic size: 1000x1004 pixels


Projecting images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 543/543 [00:01<00:00, 478.23it/s]

Orthomosaic saved to outputs/dataset1_no_gcp/dataset1_no_gcp.tif
Pipeline Complete


In [30]:
# Create pipeline with GCPs
pipeline1_with_gcp = OrthomosaicPipeline(
    image_dir=str(dataset1_dir),
    output_dir=str(output_dir / "dataset1_with_gcp"),
    feature_detector="sift",
    max_features=5000,
    match_ratio=0.7,
    use_gcps=True,
    gcp_parser=gcp_parser,
    dji_metadata=dji_metadata1
)

print("Processing dataset 1 (with GCPs)...")
output1_with_gcp = pipeline1_with_gcp.run_full_pipeline(output_name="dataset1_with_gcp")


Processing dataset 1 (with GCPs)...
Starting Orthomosaic Pipeline
Found 543 images


Loading images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 543/543 [01:14<00:00,  7.33it/s]


Loaded 543 images
Loading features from cache: outputs/dataset1_with_gcp/cache/features.pkl
Loaded features for 543 images
Loading matches from cache: outputs/dataset1_with_gcp/cache/matches.pkl
Loaded 100 matches
Estimating initial camera poses...
Initialized poses for 2 cameras
Performing bundle adjustment...
Bundle adjustment: 1905 3D points, 3810 observations, 2 cameras
Initial mean reprojection error: 0.33 pixels
Note: Using simplified bundle adjustment. For production, consider OpenSfM or COLMAP.

Reprojection Errors:
  Mean: 0.33 pixels
  Median: 0.06 pixels
  Max: 382.70 pixels
Creating orthomosaic (resolution: 0.1m/pixel)...
Orthomosaic size: 1000x1004 pixels


Projecting images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 543/543 [00:01<00:00, 479.09it/s]

Orthomosaic saved to outputs/dataset1_with_gcp/dataset1_with_gcp.tif
Pipeline Complete


## 3. Visualize Results for Dataset 1


In [31]:
# Visualize feature matches
if len(pipeline1_no_gcp.matches) > 0:
    visualize_matches(
        pipeline1_no_gcp,
        match_idx=0,
        output_path=str(output_dir / "dataset1_match_example.png"),
        max_matches=100
    )

# Visualize reprojection errors
visualize_reprojection_errors(
    pipeline1_no_gcp,
    output_path=str(output_dir / "dataset1_reprojection_errors.png")
)

# Visualize camera poses
visualize_camera_poses(
    pipeline1_no_gcp,
    output_path=str(output_dir / "dataset1_camera_poses.png")
)

# Create match quality report
create_match_quality_report(
    pipeline1_no_gcp,
    output_dir=str(output_dir / "dataset1_matches")
)


Match visualization saved to outputs/dataset1_match_example.png
Reprojection error visualization saved to outputs/dataset1_reprojection_errors.png
Camera pose visualization saved to outputs/dataset1_camera_poses.png
Creating match quality report...
Match visualization saved to outputs/dataset1_matches/match_visualization_1.png
Match visualization saved to outputs/dataset1_matches/match_visualization_2.png
Match visualization saved to outputs/dataset1_matches/match_visualization_3.png
Match visualization saved to outputs/dataset1_matches/match_visualization_4.png
Match visualization saved to outputs/dataset1_matches/match_visualization_5.png
Match visualization saved to outputs/dataset1_matches/match_visualization_worst_96.png
Match visualization saved to outputs/dataset1_matches/match_visualization_worst_97.png
Match visualization saved to outputs/dataset1_matches/match_visualization_worst_98.png
Match visualization saved to outputs/dataset1_matches/match_visualization_worst_99.png
Mat

## 4. Process Second Dataset (DJI_202510060955_019_25-3288)


In [32]:
# Setup for second dataset
dataset2_dir = data_dir / "DJI_202510060955_019_25-3288"

# Try to parse DJI metadata
dji_metadata2 = DJIMetadataParser(str(dataset2_dir))

# Create pipeline without GCPs
pipeline2_no_gcp = OrthomosaicPipeline(
    image_dir=str(dataset2_dir),
    output_dir=str(output_dir / "dataset2_no_gcp"),
    feature_detector="sift",
    max_features=5000,
    match_ratio=0.7,
    use_gcps=False,
    dji_metadata=dji_metadata2
)

print("Processing dataset 2 (without GCPs)...")
output2_no_gcp = pipeline2_no_gcp.run_full_pipeline(output_name="dataset2_no_gcp")


Parsing timestamp file: /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25/DJI_202510060955_019_25-3288/DJI_202510060955_019_Timestamp.MRK
Parsing navigation file: /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25/DJI_202510060955_019_25-3288/DJI_202510060955_019_PPKNAV.nav
Processing dataset 2 (without GCPs)...
Starting Orthomosaic Pipeline
Found 528 images


Loading images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 528/528 [01:08<00:00,  7.66it/s]


Loaded 528 images
Loading features from cache: outputs/dataset2_no_gcp/cache/features.pkl
Loaded features for 528 images
Loading matches from cache: outputs/dataset2_no_gcp/cache/matches.pkl
Loaded 100 matches
Estimating initial camera poses...
Initialized poses for 2 cameras
Performing bundle adjustment...
Bundle adjustment: 10 3D points, 20 observations, 2 cameras
Initial mean reprojection error: 894.77 pixels
Note: Using simplified bundle adjustment. For production, consider OpenSfM or COLMAP.

Reprojection Errors:
  Mean: 894.77 pixels
  Median: 0.00 pixels
  Max: 8186.22 pixels
Creating orthomosaic (resolution: 0.1m/pixel)...
Orthomosaic size: 1000x1000 pixels


Projecting images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 528/528 [00:01<00:00, 436.73it/s]

Orthomosaic saved to outputs/dataset2_no_gcp/dataset2_no_gcp.tif
Pipeline Complete


In [33]:
# Create pipeline with GCPs
pipeline2_with_gcp = OrthomosaicPipeline(
    image_dir=str(dataset2_dir),
    output_dir=str(output_dir / "dataset2_with_gcp"),
    feature_detector="sift",
    max_features=5000,
    match_ratio=0.7,
    use_gcps=True,
    gcp_parser=gcp_parser,
    dji_metadata=dji_metadata2
)

print("Processing dataset 2 (with GCPs)...")
output2_with_gcp = pipeline2_with_gcp.run_full_pipeline(output_name="dataset2_with_gcp")


Processing dataset 2 (with GCPs)...
Starting Orthomosaic Pipeline
Found 528 images


Loading images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 528/528 [01:08<00:00,  7.73it/s]


Loaded 528 images
Loading features from cache: outputs/dataset2_with_gcp/cache/features.pkl
Loaded features for 528 images
Loading matches from cache: outputs/dataset2_with_gcp/cache/matches.pkl
Loaded 100 matches
Estimating initial camera poses...
Initialized poses for 2 cameras
Performing bundle adjustment...
Bundle adjustment: 10 3D points, 20 observations, 2 cameras
Initial mean reprojection error: 894.77 pixels
Note: Using simplified bundle adjustment. For production, consider OpenSfM or COLMAP.

Reprojection Errors:
  Mean: 894.77 pixels
  Median: 0.00 pixels
  Max: 8186.22 pixels
Creating orthomosaic (resolution: 0.1m/pixel)...
Orthomosaic size: 1000x1000 pixels


Projecting images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 528/528 [00:01<00:00, 467.01it/s]

Orthomosaic saved to outputs/dataset2_with_gcp/dataset2_with_gcp.tif
Pipeline Complete


## 5. Download Basemap for Comparison


In [1]:
# Convert UTM bounds to lat/lon for basemap download
import utm
from pathlib import Path

# Import required modules if not already imported
try:
    _ = GCPParser
    _ = download_basemap
except NameError:
    import sys
    sys.path.insert(0, str(Path.cwd()))
    from westminster_ground_truth_analysis import GCPParser, download_basemap

# Get GCP bounds if not already defined
try:
    # Check if bounds are already defined
    _ = min_x, min_y, max_x, max_y
    print("Using existing GCP bounds")
except NameError:
    # Get bounds from GCP parser (create parser if needed)
    try:
        _ = gcp_parser
    except NameError:
        # Create GCP parser if it doesn't exist
        # Check if data_dir is defined, if not use default path
        try:
            _ = data_dir
        except NameError:
            data_dir = Path("/Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25")
            print(f"Using default data_dir: {data_dir}")
        
        gcp_file = data_dir / "25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv"
        gcp_parser = GCPParser(str(gcp_file))
        print(f"Created GCP parser from {gcp_file}")
    
    # Get bounds from GCP parser
    min_x, min_y, max_x, max_y = gcp_parser.get_bounds()
    print(f"Retrieved GCP bounds: X=[{min_x:.2f}, {max_x:.2f}], Y=[{min_y:.2f}, {max_y:.2f}]")

# Convert GCP bounds from UTM to lat/lon
# UTM Zone 10N (based on GCP file name)
# NOTE: In the CSV, X column is actually Northing (~5.45M) and Y column is Easting (~500k)
# utm.to_latlon expects (easting, northing), so we need to swap them
center_easting = (min_y + max_y) / 2  # Y column is easting
center_northing = (min_x + max_x) / 2  # X column is northing

# Convert center point
lat_center, lon_center = utm.to_latlon(center_easting, center_northing, 10, 'N')

# Approximate bounds in lat/lon (rough conversion)
# For more accuracy, convert all corners
# Swap X and Y: Y is easting, X is northing
lat_min, lon_min = utm.to_latlon(min_y, min_x, 10, 'N')
lat_max, lon_max = utm.to_latlon(max_y, max_x, 10, 'N')

bbox = (min(lat_min, lat_max), min(lon_min, lon_max), 
        max(lat_min, lat_max), max(lon_min, lon_max))

print(f"Basemap bounding box: {bbox}")

# Check if output_dir is defined, if not use default
try:
    _ = output_dir
except NameError:
    output_dir = Path("outputs")
    output_dir.mkdir(exist_ok=True)
    print(f"Using default output_dir: {output_dir}")

# Download basemap
basemap_path = download_basemap(
    bbox=bbox,
    output_path=str(output_dir / "basemap.tif"),
    source="esri_world_imagery",
    target_resolution=0.1  # 0.1m per pixel
)


Using default data_dir: /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25
Created GCP parser from /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25/25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv
Retrieved GCP bounds: X=[5450109.82, 5450992.66], Y=[506577.77, 507315.01]
Basemap bounding box: (np.float64(49.20374809577927), np.float64(-122.90970020449743), np.float64(49.21168116673544), np.float64(-122.89956337139125))
Using default output_dir: outputs
Tile range: X [41571, 41579], Y [89790, 89799]
Basemap saved to outputs/basemap.tif


## 6. Compare Orthomosaics to Basemap


In [2]:
# Import compare_orthomosaic_to_basemap if not already imported
try:
    _ = compare_orthomosaic_to_basemap
except NameError:
    import sys
    from pathlib import Path
    sys.path.insert(0, str(Path.cwd()))
    from westminster_ground_truth_analysis import compare_orthomosaic_to_basemap

# Check if output variables are defined, if not try to find output files
try:
    _ = output1_no_gcp
except NameError:
    # Try to find the output file
    try:
        _ = output_dir
    except NameError:
        output_dir = Path("outputs")
    
    output1_no_gcp = output_dir / "dataset1_no_gcp" / "dataset1_no_gcp.tif"
    if not output1_no_gcp.exists():
        raise FileNotFoundError(f"Orthomosaic file not found: {output1_no_gcp}. Please run the pipeline first.")
    print(f"Found orthomosaic: {output1_no_gcp}")

try:
    _ = output1_with_gcp
except NameError:
    output1_with_gcp = output_dir / "dataset1_with_gcp" / "dataset1_with_gcp.tif"
    if not output1_with_gcp.exists():
        print(f"Warning: Orthomosaic with GCPs not found: {output1_with_gcp}")

try:
    _ = basemap_path
except NameError:
    basemap_path = output_dir / "basemap.tif"
    if not basemap_path.exists():
        raise FileNotFoundError(f"Basemap file not found: {basemap_path}. Please download basemap first.")

# Compare dataset 1 without GCPs
print("=" * 60)
print("Dataset 1 - Without GCPs")
print("=" * 60)
metrics1_no_gcp = compare_orthomosaic_to_basemap(
    str(output1_no_gcp),
    str(basemap_path),
    output_dir=str(output_dir / "comparison1_no_gcp")
)

# Compare dataset 1 with GCPs
print("\n" + "=" * 60)
print("Dataset 1 - With GCPs")
print("=" * 60)
metrics1_with_gcp = compare_orthomosaic_to_basemap(
    str(output1_with_gcp),
    str(basemap_path),
    output_dir=str(output_dir / "comparison1_with_gcp")
)


Found orthomosaic: outputs/dataset1_no_gcp/dataset1_no_gcp.tif
Dataset 1 - Without GCPs
Comparing orthomosaic to basemap...
Using feature matching approach for 2D displacement calculation...
Computing 2D displacement via feature matching...
Resized basemap to 1669x2000 for matching
Detecting features in orthomosaic and basemap...
  Found 0 features in orthomosaic, 5000 features in basemap
Feature matching could not find enough correspondences between orthomosaic and basemap.

Dataset 1 - With GCPs
Comparing orthomosaic to basemap...
Using feature matching approach for 2D displacement calculation...
Computing 2D displacement via feature matching...
Resized basemap to 1669x2000 for matching
Detecting features in orthomosaic and basemap...
  Found 0 features in orthomosaic, 5000 features in basemap
Feature matching could not find enough correspondences between orthomosaic and basemap.


In [3]:
# Check if dataset 2 output variables are defined
try:
    _ = output2_no_gcp
except NameError:
    output2_no_gcp = output_dir / "dataset2_no_gcp" / "dataset2_no_gcp.tif"
    if not output2_no_gcp.exists():
        print(f"Warning: Dataset 2 orthomosaic without GCPs not found: {output2_no_gcp}")

try:
    _ = output2_with_gcp
except NameError:
    output2_with_gcp = output_dir / "dataset2_with_gcp" / "dataset2_with_gcp.tif"
    if not output2_with_gcp.exists():
        print(f"Warning: Dataset 2 orthomosaic with GCPs not found: {output2_with_gcp}")

# Compare dataset 2 without GCPs (only if file exists)
if output2_no_gcp.exists():
    print("=" * 60)
    print("Dataset 2 - Without GCPs")
    print("=" * 60)
    metrics2_no_gcp = compare_orthomosaic_to_basemap(
        str(output2_no_gcp),
        str(basemap_path),
        output_dir=str(output_dir / "comparison2_no_gcp")
    )
else:
    print("Skipping Dataset 2 - Without GCPs (file not found)")

# Compare dataset 2 with GCPs (only if file exists)
if output2_with_gcp.exists():
    print("\n" + "=" * 60)
    print("Dataset 2 - With GCPs")
    print("=" * 60)
    metrics2_with_gcp = compare_orthomosaic_to_basemap(
        str(output2_with_gcp),
        str(basemap_path),
        output_dir=str(output_dir / "comparison2_with_gcp")
    )
else:
    print("Skipping Dataset 2 - With GCPs (file not found)")


Dataset 2 - Without GCPs
Comparing orthomosaic to basemap...
Using feature matching approach for 2D displacement calculation...
Computing 2D displacement via feature matching...
Resized basemap to 1669x2000 for matching
Detecting features in orthomosaic and basemap...
  Found 0 features in orthomosaic, 5000 features in basemap
Feature matching could not find enough correspondences between orthomosaic and basemap.

Dataset 2 - With GCPs
Comparing orthomosaic to basemap...
Using feature matching approach for 2D displacement calculation...
Computing 2D displacement via feature matching...
Resized basemap to 1669x2000 for matching
Detecting features in orthomosaic and basemap...
  Found 0 features in orthomosaic, 5000 features in basemap
Feature matching could not find enough correspondences between orthomosaic and basemap.


## 7. Summary and Comparison


In [ ]:
# Create comparison summary
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Helper function to extract metric value, handling both feature matching and pixel-based metrics
def get_metric(metrics_dict, key, fallback_key=None):
    """Extract metric value, trying primary key first, then fallback."""
    if key in metrics_dict:
        return metrics_dict[key]
    if fallback_key and fallback_key in metrics_dict:
        return metrics_dict[fallback_key]
    # Check pixel_based sub-dict
    if 'pixel_based' in metrics_dict and isinstance(metrics_dict['pixel_based'], dict):
        if key in metrics_dict['pixel_based']:
            return metrics_dict['pixel_based'][key]
    return None

# Check if metrics are defined, if not try to find them or create empty placeholders
def get_metrics_or_none(var_name):
    """Try to get metrics variable, return None if not defined."""
    try:
        return globals()[var_name]
    except KeyError:
        return None

# Collect all metrics (use None if not defined)
metrics1_no_gcp = get_metrics_or_none('metrics1_no_gcp')
metrics1_with_gcp = get_metrics_or_none('metrics1_with_gcp')
metrics2_no_gcp = get_metrics_or_none('metrics2_no_gcp')
metrics2_with_gcp = get_metrics_or_none('metrics2_with_gcp')

all_metrics = [
    metrics1_no_gcp,
    metrics1_with_gcp,
    metrics2_no_gcp,
    metrics2_with_gcp
]

# Check if any metrics are available
if all(m is None for m in all_metrics):
    print("Warning: No metrics found. Please run the comparison cells first.")
    print("Creating empty summary with placeholder values.")
    # Create empty metrics dictionaries
    empty_metrics = {
        'displacement_x_pixels': 0.0,
        'displacement_y_pixels': 0.0,
        'displacement_magnitude_pixels': 0.0,
        'num_matches': 0,
        'rmse_pixels': 0.0,
        'note': 'Metrics not available - run comparison cells first'
    }
    all_metrics = [empty_metrics] * 4
    metrics1_no_gcp = metrics1_with_gcp = metrics2_no_gcp = metrics2_with_gcp = empty_metrics
else:
    # Replace None with empty dict
    all_metrics = [m if m is not None else {} for m in all_metrics]

# Build summary data, handling both metric types
summary_data = {
    'Dataset': ['Dataset 1', 'Dataset 1', 'Dataset 2', 'Dataset 2'],
    'GCPs Used': ['No', 'Yes', 'No', 'Yes'],
}

# Try to get pixel-based metrics first, fallback to feature matching metrics
summary_data['RMSE'] = [
    get_metric(m, 'rmse', 'rmse_pixels') or 0.0
    for m in all_metrics
]

summary_data['MAE'] = [
    get_metric(m, 'mae') or 0.0
    for m in all_metrics
]

summary_data['Correlation'] = [
    get_metric(m, 'correlation') or 0.0
    for m in all_metrics
]

summary_data['SSIM'] = [
    get_metric(m, 'ssim') or 0.0
    for m in all_metrics
]

# Add displacement metrics if available (from feature matching)
if any('displacement_magnitude_pixels' in m for m in all_metrics):
    summary_data['Displacement (pixels)'] = [
        get_metric(m, 'displacement_magnitude_pixels') or 0.0
        for m in all_metrics
    ]
    
if any('displacement_magnitude_meters' in m and m.get('displacement_magnitude_meters') is not None for m in all_metrics):
    summary_data['Displacement (meters)'] = [
        get_metric(m, 'displacement_magnitude_meters') or 0.0
        for m in all_metrics
    ]

# Add number of matches if available
if any('num_matches' in m for m in all_metrics):
    summary_data['Num Matches'] = [
        get_metric(m, 'num_matches') or 0
        for m in all_metrics
    ]

df = pd.DataFrame(summary_data)
print("\n" + "=" * 60)
print("SUMMARY COMPARISON")
print("=" * 60)
print(df.to_string(index=False))

# Visualize comparison
# Determine which metrics are available and non-zero
available_metrics = [col for col in df.columns if col not in ['Dataset', 'GCPs Used'] and df[col].sum() > 0]

# Create subplots based on available metrics
n_metrics = len(available_metrics)
if n_metrics == 0:
    print("No metrics available for visualization")
else:
    n_cols = 2
    n_rows = (n_metrics + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 5 * n_rows))
    if n_metrics == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    for idx, metric in enumerate(available_metrics):
        ax = axes[idx]
        x = np.arange(2)  # Two datasets
        width = 0.35
        
        no_gcp_values = df[df['GCPs Used'] == 'No'][metric].values
        with_gcp_values = df[df['GCPs Used'] == 'Yes'][metric].values
        
        # Only plot if we have values for both datasets
        if len(no_gcp_values) >= 2 and len(with_gcp_values) >= 2:
            ax.bar(x - width/2, no_gcp_values[:2], width, label='Without GCPs', alpha=0.7)
            ax.bar(x + width/2, with_gcp_values[:2], width, label='With GCPs', alpha=0.7)
            
            ax.set_xlabel('Dataset')
            ax.set_ylabel(metric)
            ax.set_title(f'{metric} Comparison')
            ax.set_xticks(x)
            ax.set_xticklabels(['Dataset 1', 'Dataset 2'])
            ax.legend()
            ax.grid(True, alpha=0.3, axis='y')
        else:
            ax.text(0.5, 0.5, f'Insufficient data for {metric}', 
                   ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{metric} Comparison')
    
    # Hide unused subplots
    for idx in range(n_metrics, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    
    # Check if output_dir is defined
    try:
        _ = output_dir
    except NameError:
        output_dir = Path("outputs")
        output_dir.mkdir(exist_ok=True)
    
    plt.savefig(output_dir / "accuracy_comparison.png", dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nComparison visualization saved to {output_dir / 'accuracy_comparison.png'}")

# Convert all .tif files to PNG for easier browser viewing
print("\n" + "=" * 60)
print("Diagnosing TIF files before conversion...")
print("=" * 60)

# First, diagnose the TIF files to see what's in them
try:
    _ = output_dir
except NameError:
    output_dir = Path("outputs")

import rasterio

# Find all .tif files in output directories
tif_files = list(output_dir.rglob("*.tif")) + list(output_dir.rglob("*.TIF"))

# Filter out checkpoint files
tif_files = [f for f in tif_files if '.ipynb_checkpoints' not in str(f)]

if tif_files:
    print(f"Found {len(tif_files)} .tif file(s) to diagnose:")
    for tif_path in tif_files:
        try:
            with rasterio.open(tif_path) as src:
                data = src.read()
                print(f"\n  {tif_path.relative_to(output_dir)}:")
                print(f"    Shape: {data.shape}")
                print(f"    Dtype: {data.dtype}")
                print(f"    Min: {data.min()}, Max: {data.max()}")
                print(f"    Mean: {data.mean():.2f}, Std: {data.std():.2f}")
                
                # Check if data is all zeros or very dark
                if data.max() == 0:
                    print(f"    ⚠️  WARNING: All pixels are zero (black image)")
                elif data.max() < 10:
                    print(f"    ⚠️  WARNING: Very dark image (max value: {data.max()})")
                elif data.max() <= 1.0 and data.dtype in [np.float32, np.float64]:
                    print(f"    ⚠️  WARNING: Data appears to be normalized to [0,1] range")
                    print(f"    Suggestion: Multiply by 255 for display")
                else:
                    print(f"    ✓ Data looks reasonable")
                    
                # Check non-zero pixels
                non_zero = np.count_nonzero(data)
                total_pixels = data.size
                print(f"    Non-zero pixels: {non_zero}/{total_pixels} ({100*non_zero/total_pixels:.1f}%)")
        except Exception as e:
            print(f"  Error reading {tif_path.name}: {e}")

print("\n" + "=" * 60)
print("Converting all .tif files to PNG format...")
print("=" * 60)

try:
    _ = output_dir
except NameError:
    output_dir = Path("outputs")

import rasterio
from PIL import Image

def convert_tif_to_png(tif_path: Path, png_path: Path, max_size: int = 4000):
    """Convert a GeoTIFF to PNG format, handling different band configurations."""
    try:
        with rasterio.open(tif_path) as src:
            # Read all bands
            data = src.read()
            
            # Get data type info
            dtype = data.dtype
            data_min, data_max = data.min(), data.max()
            print(f"    Data type: {dtype}, Shape: {data.shape}, Range: [{data_min:.2f}, {data_max:.2f}]")
            
            # If data is already uint8 and in valid range, use directly
            img = None
            if dtype == np.uint8 and data_max <= 255 and data_min >= 0:
                # Data is already in correct format, just need to reshape
                if len(data.shape) == 3 and data.shape[0] == 3:
                    img_data = np.transpose(data, (1, 2, 0))
                    img = Image.fromarray(img_data, mode='RGB')
                elif len(data.shape) == 3 and data.shape[0] == 1:
                    img = Image.fromarray(data[0], mode='L')
                elif len(data.shape) == 2:
                    img = Image.fromarray(data, mode='L')
            
            # Check if data is normalized to [0,1] range (common in some workflows)
            elif dtype in [np.float32, np.float64] and data_max <= 1.0 and data_min >= 0.0:
                print(f"    Note: Data appears normalized to [0,1], scaling to [0,255]")
                # Scale to 0-255
                if len(data.shape) == 3 and data.shape[0] == 3:
                    img_data = (np.transpose(data, (1, 2, 0)) * 255).astype(np.uint8)
                    img = Image.fromarray(img_data, mode='RGB')
                elif len(data.shape) == 3 and data.shape[0] == 1:
                    img = Image.fromarray((data[0] * 255).astype(np.uint8), mode='L')
                elif len(data.shape) == 2:
                    img = Image.fromarray((data * 255).astype(np.uint8), mode='L')
            
            # If we didn't create img from uint8 data, do normalization
            if img is None:
                # Convert to float for processing
                if dtype != np.float32 and dtype != np.float64:
                    data = data.astype(np.float32)
                
                # Handle different band configurations
                if len(data.shape) == 2:
                    # Single band - convert to grayscale
                    img_data = data.copy()
                    # Use percentile-based normalization to handle outliers
                    p2, p98 = np.percentile(img_data[img_data > 0] if (img_data > 0).any() else img_data, [2, 98])
                    if p98 > p2:
                        img_data = np.clip((img_data - p2) / (p98 - p2) * 255, 0, 255).astype(np.uint8)
                    else:
                        img_data = np.clip(img_data, 0, 255).astype(np.uint8)
                    img = Image.fromarray(img_data, mode='L')
                elif len(data.shape) == 3:
                    # Multi-band
                    if data.shape[0] == 1:
                        # Single band in 3D array
                        img_data = data[0].copy()
                        p2, p98 = np.percentile(img_data[img_data > 0] if (img_data > 0).any() else img_data, [2, 98])
                        if p98 > p2:
                            img_data = np.clip((img_data - p2) / (p98 - p2) * 255, 0, 255).astype(np.uint8)
                        else:
                            img_data = np.clip(img_data, 0, 255).astype(np.uint8)
                        img = Image.fromarray(img_data, mode='L')
                    elif data.shape[0] == 3:
                        # RGB
                        img_data = np.transpose(data, (1, 2, 0)).copy()
                        # Normalize each band using percentiles
                        for i in range(3):
                            band = img_data[:, :, i]
                            # Use percentile-based normalization
                            p2, p98 = np.percentile(band[band > 0] if (band > 0).any() else band, [2, 98])
                            if p98 > p2:
                                img_data[:, :, i] = np.clip((band - p2) / (p98 - p2) * 255, 0, 255).astype(np.uint8)
                            else:
                                img_data[:, :, i] = np.clip(band, 0, 255).astype(np.uint8)
                        img = Image.fromarray(img_data, mode='RGB')
                    elif data.shape[0] == 4:
                        # RGBA
                        img_data = np.transpose(data, (1, 2, 0)).copy()
                        for i in range(4):
                            band = img_data[:, :, i]
                            p2, p98 = np.percentile(band[band > 0] if (band > 0).any() else band, [2, 98])
                            if p98 > p2:
                                img_data[:, :, i] = np.clip((band - p2) / (p98 - p2) * 255, 0, 255).astype(np.uint8)
                            else:
                                img_data[:, :, i] = np.clip(band, 0, 255).astype(np.uint8)
                        img = Image.fromarray(img_data, mode='RGBA')
                    else:
                        # Multiple bands - use first 3 for RGB
                        img_data = np.transpose(data[:3], (1, 2, 0)).copy()
                        for i in range(3):
                            band = img_data[:, :, i]
                            p2, p98 = np.percentile(band[band > 0] if (band > 0).any() else band, [2, 98])
                            if p98 > p2:
                                img_data[:, :, i] = np.clip((band - p2) / (p98 - p2) * 255, 0, 255).astype(np.uint8)
                            else:
                                img_data[:, :, i] = np.clip(band, 0, 255).astype(np.uint8)
                        img = Image.fromarray(img_data, mode='RGB')
                else:
                    print(f"  Warning: Unsupported data shape {data.shape} for {tif_path.name}")
                    return False
            
            # Resize if too large (for both uint8 direct and normalized images)
            width, height = img.size
            if width > max_size or height > max_size:
                scale = min(max_size / width, max_size / height)
                new_width = int(width * scale)
                new_height = int(height * scale)
                img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                print(f"  Resized {tif_path.name} from {width}x{height} to {new_width}x{new_height}")
            
            # Save PNG
            png_path.parent.mkdir(parents=True, exist_ok=True)
            img.save(png_path, 'PNG', optimize=True)
            return True
            
    except Exception as e:
        print(f"  Error converting {tif_path.name}: {e}")
        return False

# Find all .tif files in output directories
tif_files = list(output_dir.rglob("*.tif")) + list(output_dir.rglob("*.TIF"))

if not tif_files:
    print(f"No .tif files found in {output_dir}")
else:
    print(f"Found {len(tif_files)} .tif file(s) to convert")
    
    converted = 0
    skipped = 0
    
    for tif_path in tif_files:
        # Skip checkpoint files
        if '.ipynb_checkpoints' in str(tif_path):
            print(f"  Skipping {tif_path.relative_to(output_dir)} (checkpoint file)")
            skipped += 1
            continue
        
        png_path = tif_path.with_suffix('.png')
        
        # Force regenerate PNGs to ensure they use the latest conversion logic
        # (especially important after fixing the black image issue)
        force_regenerate = True
        
        if png_path.exists() and not force_regenerate:
            # Only skip if PNG is newer and we're not forcing regeneration
            if png_path.stat().st_mtime > tif_path.stat().st_mtime:
                print(f"  Skipping {tif_path.relative_to(output_dir)} (PNG already exists and is newer)")
                skipped += 1
                continue
        
        print(f"  Converting {tif_path.relative_to(output_dir)}...")
        if convert_tif_to_png(tif_path, png_path):
            converted += 1
            print(f"    ✓ Saved to {png_path.relative_to(output_dir)}")
        else:
            skipped += 1
    
    print(f"\nConversion complete: {converted} converted, {skipped} skipped")



SUMMARY COMPARISON
  Dataset GCPs Used  RMSE  MAE  Correlation  SSIM  Displacement (pixels)  Num Matches
Dataset 1        No   0.0  0.0          0.0   0.0                    0.0            0
Dataset 1       Yes   0.0  0.0          0.0   0.0                    0.0            0
Dataset 2        No   0.0  0.0          0.0   0.0                    0.0            0
Dataset 2       Yes   0.0  0.0          0.0   0.0                    0.0            0
No metrics available for visualization

Converting all .tif files to PNG format...
Found 6 .tif file(s) to convert
  Converting basemap.tif...
    Data type: uint8, Shape: (3, 2263, 1889), Range: [0.00, 255.00]


/var/folders/45/msyhv5rs7xb6v19f926jmt_h0000gp/T/ipykernel_97142/3337865715.py:204: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(img_data, mode='RGB')


    ✓ Saved to basemap.png
  Converting dataset2_no_gcp/dataset2_no_gcp.tif...
    Data type: uint8, Shape: (3, 1000, 1000), Range: [0.00, 107.00]
    ✓ Saved to dataset2_no_gcp/dataset2_no_gcp.png
  Converting dataset1_with_gcp/dataset1_with_gcp.tif...
    Data type: uint8, Shape: (3, 1004, 1000), Range: [0.00, 0.00]
    ✓ Saved to dataset1_with_gcp/dataset1_with_gcp.png
  Converting dataset2_with_gcp/dataset2_with_gcp.tif...
    Data type: uint8, Shape: (3, 1000, 1000), Range: [0.00, 107.00]
    ✓ Saved to dataset2_with_gcp/dataset2_with_gcp.png
  Converting dataset1_no_gcp/dataset1_no_gcp.tif...
    Data type: uint8, Shape: (3, 1004, 1000), Range: [0.00, 0.00]
    ✓ Saved to dataset1_no_gcp/dataset1_no_gcp.png
  Skipping dataset1_no_gcp/.ipynb_checkpoints/dataset1_no_gcp-checkpoint.tif (checkpoint file)

Conversion complete: 5 converted, 1 skipped
